In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from string import ascii_letters
import re
from datetime import datetime
from time import time
from tqdm import tqdm
import os
import multiprocessing
from multiprocessing import Pool
from pyspark import SparkContext, SparkConf

In [15]:
print('> file management func:\n')

> file management func:



In [25]:
def get_content(directory):
    content = []
    for file in os.scandir(directory):
        if file.is_dir():
            content.append(file.path + '/')
        if file.is_file():
            content.append(file.path)
    return content
    
print('    get_content(directory)')

    get_content(directory)


In [8]:
def get_last_update(file_path):

    last_modified_timestamp = os.path.getmtime(file_path)
    last_modified_datetime = datetime.fromtimestamp(last_modified_timestamp)
    return last_modified_datetime.strftime("%m-%d-%Y")

print('    get_last_update(file_path)')

    get_last_update(file_path)


In [139]:
print('\n> Updating lists func:\n')


> Updating lists func:



In [137]:
def get_participants(event_link):
    
    xml = get_xml(url=event_link)
    
    iterator = iter(xml.find_all('a',{'class':'b-link b-link_style_black'}))
    participants = []
    links = []
    
    for item in iterator:
        participants.append(item.text.strip())
        links.append(item['href'])
    
    return list(zip(participants,links))

print('    get_participants(event_link)')

    get_event_data(event_link)


In [18]:
print('\n> fighters_stats_scrape func:\n')


> fighters_stats_scrape func:



In [3]:
def get_xml(url, data_type='xml', headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}):
    
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, data_type)
    except:
        #print('Error on: ' + url) 
        return None
    
    return soup

print("    get_xml(url, data_type='xml', headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.")

    get_xml(url, data_type='xml', headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.


In [1]:
def get_links(url):
    
    link_items =  get_xml(url=url).find_all('a',{'class':'b-link b-link_style_black'})
    dates_items = get_xml(url=url).find_all('span',{'class':'b-statistics__date'})
    
    links = []
    dates = []
    for url,date in zip(link_items,dates_items):
        links.append(url['href'])
        dates.append(datetime.strptime(date.text.strip(), "%B %d, %Y").strftime('%m-%d-%Y'))
    
    data = pd.DataFrame(data = zip(dates,links), columns=['date','event_url'])
    data.date = pd.to_datetime(data['date'], format='%m-%d-%Y')
    return data
    
print('    get_links(url)')

    get_links(url)


In [130]:
def get_fighter_data(url):
    
    xml =  get_xml(url=url)
    data = []
    data.append(xml.find('h2').find('span',{'class':'b-content__title-highlight'}).text.strip()) #name
    
    record_string = xml.find('h2').find('span',{'class':'b-content__title-record'}).text.strip()
    data.append(int(record_string[record_string.find(':')+1:record_string.find('-')])) #wins
    data.append(int(record_string.split('-')[1])) #losses
    data.append(record_string.split('-')[2]) #draws
    
    stat_boxes = xml.find_all('ul')[0:3]
    for item in stat_boxes:
        for sub_item in item.contents:
            text = sub_item.text.strip().replace(" ", "").replace("\n", "")
            if len(text) > 0:
                data.append(text[text.find(':')+1:])
    
    data.append(get_current_date()) #last_update
    
    date_table_location = 7
    try:
        date_string = xml.find_all('td')[date_table_location].find_all('p')[1].text.strip()
        last_match = datetime.strptime(date_string, '%b. %d, %Y').strftime('%m-%d-%Y')
    except:
        last_match = None

    data.append(last_match) #last_match
    data.append(url)
    return data

print('    get_fighter_data(url) # Retrive data as list') 

    get_fighter_data(url) # Retrive data as list


In [114]:
def get_current_date():
    return datetime.today().strftime('%m-%d-%Y')
    
print('    get_current_date()')    

    get_current_date()


In [131]:
print('\n> matches_stats_web_scrape func:\n')


> matches_stats_web_scrape func:



In [97]:
def get_event_date(xml):
    
    '''return event date'''
    
    item = xml.find('li',{'class':'b-list__box-list-item'})
    string = item.get_text().strip()
    start = len('DATE:')
    date = string[start:].strip()
    
    return datetime.strptime(date, "%B %d, %Y").strftime("%m-%d-%Y")

print('    get_event_date(xml)')

    get_event_date(xml)


In [64]:
def get_event_data(event_link):
    
    xml = get_xml(url=event_link)
    #num_of_matches = len(xml.find_all('i',{'class':'b-flag__text'}))
    
    iterator = iter(xml.find_all('p',{'class':'b-fight-details__table-text'}))
    data = []
    start_flag = True
    row = []
    
    while True:
        
        try:
            item = next(iterator)
            text_item = item.text.strip().replace('\n','')
            
            if text_item == 'draw' or text_item == 'nc': # begining of new row 
                
                if start_flag == False:
                    row.append(get_event_date(xml=xml))
                    row.append(event_link) #
                    data.append(row)
                
                start_flag = False
                row = []
                row.append(text_item)
                next(iterator)
            
            elif text_item == 'win': # begining of new row 
                
                if start_flag == False:
                    row.append(get_event_date(xml=xml))
                    row.append(event_link) #
                    data.append(row)
                    
                start_flag = False
                row = []
                row.append(text_item)
            
            elif 'weight' in text_item:
                row.append(text_item[0:text_item.find('weight')+6])
            else:
                row.append(text_item)
            
        except StopIteration:
            break
    
    
    return data    
    


print('    get_event_data(event_link)')

    get_event_data(event_link)


In [12]:
def get_later_date(date1, date2):

    date1_obj = datetime.strptime(date1, "%d-%m-%Y")
    date2_obj = datetime.strptime(date2, "%d-%m-%Y")

    if date1_obj > date2_obj:
        return date1
    else:
        return date2
    
print('    get_later_date(date1, date2)')    

    get_later_date(date1, date2)


In [141]:
print('\n> upcoming_web_scrape func:\n')


> upcoming_web_scrape func:



In [143]:
def get_upcoming_event_url(xml):
    return xml.find('a',{'class':'b-link b-link_style_white'})['href']

print('    get_upcoming_event_url(xml)')

    get_upcoming_link(xml)


In [4]:
def get_upcoming_event_data(upcoming_event_url, event_name=None, event_time=None):
    
    event_xml = get_xml(url=upcoming_event_url)
    data = {'fighter':[],
            'opponent':[],
            'weight_class':[],
            'event_name':[],
            'date':[]
           }
    
    text_data = []
    for item in event_xml.find_all('p',{'class':'b-fight-details__table-text'}):
        string = item.get_text().strip()
        
        if string != 'View Matchup' and string != '':
            if 'weight' in string:
                data['weight_class'].append(string)
                data['event_name'].append(event_name)
                data['date'].append(event_time)
            elif len(data['fighter']) == len(data['opponent']):
                data['fighter'].append(string)
            else:
                data['opponent'].append(string)
        
    return data
    
print('    get_upcoming_event_data(upcoming_event_url, event_name=None, event_time=None)')    

    get_upcoming_event_data(upcoming_event_url, event_name=None, event_time=None)


In [157]:
print('\n> fighters_stats_full_update func:\n')


> fighters_stats_full_update func:



In [160]:
def get_fighters_links(page_url):
    xml = get_xml(page_url)
    link_list = []
    for item in xml.find_all('a',{'class':'b-link b-link_style_black'}):
        link_list.append(item['href'])
        
    return link_list    

print('    get_fighters_links(page_url)')

    get_fighters_links(page_url)


In [4]:
print('\n> draftking_odds func:\n')


> draftking_odds func:



In [7]:
def get_table_odds(table):

    fighter = []
    opponent = []
    fighter_odds = []
    opponent_odds = []
    for item in table.find_all('span'):
        try:
            if 'event-cell__name' in item['class']:
                if len(fighter) == len(opponent):
                    fighter.append(item.text)
                else:
                    opponent.append(item.text)
                    
            if 'sportsbook-odds' in item['class']:
                if len(fighter_odds) == len(opponent_odds):
                    fighter_odds.append(item.text)
                else:
                    opponent_odds.append(item.text)
        except:
            continue

    return pd.DataFrame(data={'fighter':fighter,
                              'fighter_odds':fighter_odds,
                              'opponent':opponent,
                              'opponent_odds':opponent_odds})

print('    get_table_odds(table)')

    get_table_odds(table)


In [3]:
def get_odds(data, fighter, opponent):
    
    for i in range(len(data)):
        if (fighter == data.loc[i,'fighter']) and (opponent == data.loc[i,'opponent']):
            return {fighter:data.loc[i,'fighter_odds'],
                    opponent:data.loc[i,'opponent_odds']}
        if (fighter == data.loc[i,'opponent']) and (opponent == data.loc[i,'fighter']):
            return {opponent:data.loc[i,'fighter_odds'],
                    fighter:data.loc[i,'opponent_odds']}    
    
print('    get_odds(data, fighter, opponent)')

    get_odds(data, fighter, opponent)
